# STEP B

In [8]:
import cv2
from matplotlib import pyplot as plt
import numpy as np

def read_image_and_compute_keypoints(object_imgs, scene_imgs, detector):
    imgs_dict = {}

    for name_img in object_imgs:
        img_load = cv2.imread('models/' + name_img + '.jpg', cv2.IMREAD_GRAYSCALE)
        
        """
        # Enlever le centre de l'image
        row = int(img_load.shape[0]/2)
        col = int(img_load.shape[1]/2)
        pour = 0.5
        new_color = 255 # White
        img_load[int(row - row*pour) : int(row + row*pour), int(col - col*pour): int(col + col*pour)] = new_color
        #
        """

        kp, des = detector.detectAndCompute(img_load, None)
        imgs_dict[name_img] = {'kp': kp, 'des': des, 'shape': img_load.shape}

    for name_img in scene_imgs:
        img_load = cv2.imread('scenes/' + name_img + '.png', cv2.IMREAD_GRAYSCALE)
        kp, des = detector.detectAndCompute(img_load, None)
        imgs_dict[name_img] = {'kp': kp, 'des': des, 'shape': img_load.shape}

    return imgs_dict

def matching_and_localize_objects_in_scene(imgs_dict, name_scene_img, ratio_test=0.45):
    all_scene_corners = {}
    all_good ={}
    for name in object_imgs:
        object_des, object_kp = imgs_dict[name]['des'], imgs_dict[name]['kp']
        scene_des, scene_kp = imgs_dict[name_scene_img]['des'], imgs_dict[name_scene_img]['kp']
        
        #-- matching
        matches = matcher.knnMatch(object_des, scene_des, k=2)
    
        #-- store all the good matches as per Lowe's ratio test.
        good = []
        for m,n in matches:
            if m.distance < ratio_test*n.distance:
                good.append(m)
                
        if len(good) > 30:        
            #-- Get the keypoints from the good matches
            object_good_kp = np.float32([ object_kp[m.queryIdx].pt for m in good ])
            scene_good_kp = np.float32([ scene_kp[m.trainIdx].pt for m in good ])   


            #-- Get the corners of object
            h,w = imgs_dict[name]['shape'][0:2]
            object_corners = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)

            #-- Get the homography of object/scene
            H, _ =  cv2.findHomography(object_good_kp, scene_good_kp, cv2.RANSAC, 5.0)

            #-- Get the corners of object in scene
            scene_corners = cv2.perspectiveTransform(object_corners, H)


            all_scene_corners[name] = scene_corners
            all_good[name] = good
        
    return all_scene_corners, all_good

def check_best_color_matches(all_scene_corners, name_scene_img):
    
    scene_img_bgr = cv2.imread('scenes/' + name_scene_img + '.png')
    best_all_scene_corners = {}
    
    for name in all_scene_corners.keys():

        x_min = max(int(np.min(all_scene_corners[name], axis=0)[0][0]), 0)
        y_min = max(int(np.min(all_scene_corners[name], axis=0)[0][1]), 0)
        x_max = int(np.max(all_scene_corners[name], axis=0)[0][0])
        y_max = int(np.max(all_scene_corners[name], axis=0)[0][1])
        
        object_img_bgr = cv2.imread('models/' + name + '.jpg')
        
        object_img_mean_color = object_img_bgr.mean(axis=0).mean(axis=0)
        object_in_scene_img = scene_img_bgr[y_min:y_max,x_min:x_max]
        object_in_scene_img_mean_color = object_in_scene_img.mean(axis=0).mean(axis=0)   
        color_diff = np.sqrt(np.sum([value ** 2 for value in abs(object_img_mean_color - object_in_scene_img_mean_color)]))
        
        #color_diff = (np.sum([value **2 for value in (object_img_mean_color - object_in_scene_img_mean_color)]))/4
        
        if color_diff < 60:
            best_all_scene_corners[name] = all_scene_corners[name]
        
    return best_all_scene_corners

def drawn_object_lines_in_scene(img_scene, best_all_scene_corners):
    for name in best_all_scene_corners.keys():
        scene_corners = best_all_scene_corners[name]
        img_scene = cv2.polylines(img_scene, [np.int32(scene_corners)], 
                                  isClosed=True, color=(0,255,0), thickness=5)
    return img_scene

def print_object_found(best_all_scene_corners):
    for name in best_all_scene_corners.keys():  
        x_min = max(int(np.min(best_all_scene_corners[name], axis=0)[0][0]), 0)
        y_min = max(int(np.min(best_all_scene_corners[name], axis=0)[0][1]), 0)
        x_max = int(np.max(best_all_scene_corners[name], axis=0)[0][0])
        y_max = int(np.max(best_all_scene_corners[name], axis=0)[0][1])
        
        print('Product {} - {} instance/s found:'.format(name, 1))
        print('\tInstance {} position: {}, width: {}px, height: {}px'.format(1, (x_min, y_min), (x_max-x_min), (y_max-y_min)))
    
    print('_' * 80 + '\n')

def stepA():
    
    detector = cv2.SIFT_create()
    FLANN_INDEX_KDTREE = 1

        # Defining parameters for algorithm 
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)

        # Defining search params.
        # checks=50 specifies the number of times the trees in the index should be recursively traversed.
        # Higher values gives better precision, but also takes more time
    search_params = dict(checks = 50)

        # Initializing matcher
    matcher = cv2.FlannBasedMatcher(index_params, search_params)
    #matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_FLANNBASED)
    
    imgs_dict = read_image_and_compute_keypoints(object_imgs, scene_imgs, detector)
    
    
    for scene_name in scene_imgs:
        
        all_scene_corners, all_good= matching_and_localize_objects_in_scene(imgs_dict, scene_name)
        
        best_all_scene_corners = check_best_color_matches(all_scene_corners , scene_name)
        
        
        img_scene_rgb = cv2.cvtColor(cv2.imread('scenes/' + scene_name + '.png'), cv2.COLOR_BGR2RGB)
        img_scene_rgb = drawn_object_lines_in_scene(img_scene_rgb, best_all_scene_corners)

        
        print('Scene {} :'.format(scene_name))
        plt.imshow(img_scene_rgb)
        plt.show()
        print_object_found(best_all_scene_corners)

In [48]:
#-- bank of image step B
object_imgs = ['0', '1', '11', '19', '24', '25', '26']
#object_imgs = ['0', '11']
scene_imgs = ['m1', 'm2', 'm3', 'm4', 'm5']

detector = cv2.SIFT_create()
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)
matcher = cv2.FlannBasedMatcher(index_params, search_params)

In [49]:
imgs_dict = read_image_and_compute_keypoints(object_imgs, scene_imgs, detector)

In [67]:
for name in object_imgs:
    object_des, object_kp = imgs_dict[name]['des'], imgs_dict[name]['kp']
    scene_des, scene_kp = imgs_dict['m1']['des'], imgs_dict['m1']['kp']
    
    #-- matching
    matches = matcher.knnMatch(object_des, scene_des, k=2)

    #-- store all the good matches as per Lowe's ratio test.
    good = []
    for m,n in matches:
        if m.distance < 0.5*n.distance:
            good.append(m)
    
    #-- barycenter of found object keypoint 
    object_good_kp = [ object_kp[m.queryIdx] for m in good ]
    
    object_xc = np.mean([ kp.pt[0] for kp in object_good_kp ], dtype='float')
    object_yc = np.mean([ kp.pt[1] for kp in object_good_kp ], dtype='float')
    
    # create hough space 
    # map of hough space kp_train_idx -> map name-values
    hough_space = {}

    for t_idx, q_idx in good_matches.items():
        
        #compute_entry_hough_space(kp_query[q_idx], kp_train[t_idx], query_xc, query_yc)
        
        entry = {}

        v = ((q_xc - kp_q.pt[0]), (q_yc - kp_q.pt[1]))
        scale_ratio = kp_t.size / kp_q.size
        delta_angle = kp_t.angle - kp_q.angle
        x_c = kp_t.pt[0] + scale_ratio * (np.cos(delta_angle) * v[0] - np.sin(delta_angle) * v[1])
        y_c = kp_t.pt[1] + scale_ratio * (np.sin(delta_angle) * v[0] + np.cos(delta_angle) * v[1])

        entry['x_c'] = x_c
        entry['y_c'] = y_c
        entry['scale_ratio'] = scale_ratio
        entry['delta_angle'] = delta_angle
    
    hough_space[t_idx] = compute_entry_hough_space(kp_query[q_idx], kp_train[t_idx], query_xc, query_yc)
        
        
        

NameError: name 'create_hough_space' is not defined

In [68]:
good

[<DMatch 0x7fd046adc230>,
 <DMatch 0x7fd0705cec90>,
 <DMatch 0x7fd0705d8c70>,
 <DMatch 0x7fd0705d9cd0>,
 <DMatch 0x7fd0705e1eb0>,
 <DMatch 0x7fd046af4510>,
 <DMatch 0x7fd0469bd790>,
 <DMatch 0x7fd0469bdc70>,
 <DMatch 0x7fd0469ce990>,
 <DMatch 0x7fd0469de170>,
 <DMatch 0x7fd0469de230>,
 <DMatch 0x7fd0469de290>,
 <DMatch 0x7fd0469de9f0>,
 <DMatch 0x7fd0469dec30>,
 <DMatch 0x7fd06f980250>,
 <DMatch 0x7fd046f33970>,
 <DMatch 0x7fd046b637d0>,
 <DMatch 0x7fd046b63810>,
 <DMatch 0x7fd06f9bdc90>,
 <DMatch 0x7fd06f9ab650>,
 <DMatch 0x7fd046aede90>,
 <DMatch 0x7fd046b0c590>,
 <DMatch 0x7fd046aa7f50>,
 <DMatch 0x7fd046aa6950>,
 <DMatch 0x7fd046aa6990>,
 <DMatch 0x7fd06fc2d130>,
 <DMatch 0x7fd06fc56690>,
 <DMatch 0x7fd06fc56b90>,
 <DMatch 0x7fd06fc3ba30>,
 <DMatch 0x7fd06fc3bb30>,
 <DMatch 0x7fd06fc5ddd0>,
 <DMatch 0x7fd06fc3ee90>,
 <DMatch 0x7fd06fc3eed0>,
 <DMatch 0x7fd06fb2d430>,
 <DMatch 0x7fd06fb46370>,
 <DMatch 0x7fd06fb411f0>,
 <DMatch 0x7fd06fb49850>,
 <DMatch 0x7fd06fb63190>,
 <DMatch 0x7